In [212]:
%matplotlib inline
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import IPython.display as ipd
import sklearn as sk
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.compose

In [213]:
data = pd.read_csv('datasets/bikesharing-day.csv', index_col=0)

data.dteday = pd.to_datetime(data.dteday, format='%Y-%m-%d').dt.normalize()
data.season = data.season.astype('category')
data.mnth = data.mnth.astype('category')
#data['day'] = data.dteday.dt.day.astype('category')
if ('hr' in data):
    data.hr = data.hr.astype('category')
#data.holiday = data.holiday.astype(bool)
data.weekday = data.weekday.astype('category')
#data.workingday = data.workingday.astype(bool)
data.weathersit = data.weathersit.astype('category')

In [219]:
columns = ['season','weekday','holiday','temp','hum','cnt']
projected_data = data.loc[:, columns]
train, test = sk.model_selection.train_test_split(projected_data)

ct = sk.compose.ColumnTransformer([
    ('col',sk.preprocessing.OneHotEncoder(categories='auto'),[
        'season', 'weekday'
    ])
], remainder='passthrough')

In [225]:
def regression_model(model, predict, transform=True):
    variates = train.columns.drop(predict)
    
    trainX = train.loc[:, variates]
    trainy = train.loc[:,predict]
    
    testX = test.loc[:, variates]
    testy = test.loc[:,predict]
    
    if transform:
        ct.fit(projected_data.loc[:,variates]);
        trainX = ct.transform(trainX)
        testX = ct.transform(testX)
    
    model.fit(trainX,trainy)
    predictedy = model.predict(testX)
    return pd.Series({
        'RMSE': np.sqrt(sk.metrics.mean_squared_error(testy,predictedy)),
        'MAE': sk.metrics.mean_absolute_error(testy,predictedy),
        'model': model
    }, name=type(model).__name__)

In [236]:
import sklearn.linear_model
import sklearn.svm
import sklearn.neural_network


pd.DataFrame([regression_model(m, 'cnt', transform=False) for m in [
    sk.linear_model.LinearRegression(),
    sk.svm.SVR(gamma='scale'),
    sk.neural_network.MLPRegressor(solver='lbfgs')
]]).drop(columns='model').sort_values('RMSE')

,RMSE,MAE
LinearRegression,1335.897178,1110.731191
MLPRegressor,1375.005500,1137.088230
SVR,1944.147090,1564.246169
